<a href="https://colab.research.google.com/github/bipeline-12/CSOE11---Big-Data-Analytics/blob/main/110121022_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
def load_custom_data(input_file):
    # Open the file and read its contents
    with open(input_file, 'r') as file_input:
        custom_data = file_input.readlines()
    # Split each line using ';' and create a list of transactions
    custom_transactions = [line.strip().split(';') for line in custom_data]
    return custom_transactions

def load_custom_data(input_file):
    # Open the file and read its contents
    with open(input_file, 'r') as file_input:
        custom_data = file_input.readlines()
    # Split each line using ';' and create a list of transactions
    custom_transactions = [line.strip().split(';') for line in custom_data]
    return custom_transactions

def generate_initial_candidates(transactions):
    initial_candidates = []
    for transaction in transactions:
        for item in transaction:
            if [item] not in initial_candidates:
                initial_candidates.append([item])
    initial_candidates.sort()
    return list(map(frozenset, initial_candidates))

def scan_for_frequent_items(transactions, candidates, min_support):
    subset_counts = {}
    for transaction in transactions:
        for candidate in candidates:
            if candidate.issubset(transaction):
                if not candidate in subset_counts:
                    subset_counts[candidate] = 1
                else:
                    subset_counts[candidate] += 1
    num_transactions = float(len(transactions))
    frequent_itemsets = []
    support_data = {}
    for key in subset_counts:
        support = subset_counts[key] / num_transactions
        if support >= min_support:
            frequent_itemsets.insert(0, key)
        support_data[key] = support
    return frequent_itemsets, support_data

def custom_apriori_algorithm(transactions, min_support=0.01):
    initial_candidates = generate_initial_candidates(transactions)
    frequent_itemsets_1, support_data = scan_for_frequent_items(transactions, initial_candidates, min_support)
    frequent_itemsets = [frequent_itemsets_1]
    k = 2
    while (len(frequent_itemsets[k - 2]) > 0):
        candidates_k = generate_candidates(frequent_itemsets[k - 2], k)
        frequent_itemsets_k, support_data_k = scan_for_frequent_items(transactions, candidates_k, min_support)
        support_data.update(support_data_k)
        frequent_itemsets.append(frequent_itemsets_k)
        k += 1
    return frequent_itemsets, support_data

def generate_candidates(frequent_itemsets, k):
    candidates = []
    len_frequent_itemsets = len(frequent_itemsets)
    for i in range(len_frequent_itemsets):
        for j in range(i + 1, len_frequent_itemsets):
            L1 = list(frequent_itemsets[i])[:k - 2]
            L2 = list(frequent_itemsets[j])[:k - 2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                candidates.append(frequent_itemsets[i] | frequent_itemsets[j])
    return candidates

def sort_and_insert(file):
    with open(file, 'r') as f:
        lines = f.readlines()
    custom_patterns = []
    for line in lines:
        pattern, support = line.strip().split(':')
        custom_patterns.append((pattern.strip(), support.strip()))
    custom_patterns.sort(key=lambda x: float(x[1]), reverse=True)
    with open(file, 'w') as f:
        for pattern, support in custom_patterns:
            f.write(f"{pattern.strip()} : {support.strip()}\n")

def insert_total_count_in_file(file):
    with open(file, 'r') as f:
        lines = f.readlines()
    total_count = len(lines)
    with open(file, 'w') as f:
        f.write(f"#Total count: {total_count}\n")
        for line in lines:
            f.write(line)

def run_custom_analysis(input_file):
    # Load the custom dataset
    custom_dataset = load_custom_data(input_file)

    # Apply the custom Apriori algorithm
    frequent_itemsets, support_data = custom_apriori_algorithm(custom_dataset)

    # Write patterns_1.txt
    with open('patterns_1.txt', 'w') as f:
        for itemset in frequent_itemsets[0]:
            f.write(f"{list(itemset)[0]} : {support_data[itemset]}\n")
    sort_and_insert('patterns_1.txt')
    insert_total_count_in_file('patterns_1.txt')

    # Write patterns_all.txt
    with open('patterns_all.txt', 'w') as f:
        for k_itemsets in frequent_itemsets:
            for itemset in k_itemsets:
                f.write(f"{';'.join(list(itemset))} : {support_data[itemset]}\n")
    sort_and_insert('patterns_all.txt')
    insert_total_count_in_file('patterns_all.txt')

    def generate_closed_itemsets(frequent_itemsets, support_data):
        closed_itemsets = []
        for k_itemsets in frequent_itemsets:
            for itemset in k_itemsets:
                if not any([itemset.issubset(superset) and support_data[itemset] == support_data[superset] for superset in closed_itemsets]):
                    closed_itemsets.append(itemset)
        return closed_itemsets

    # Generate custom closed itemsets and write patterns_close.txt
    closed_itemsets = generate_closed_itemsets(frequent_itemsets, support_data)
    with open('patterns_close.txt', 'w') as f:
        for itemset in closed_itemsets:
            f.write(f"{';'.join(list(itemset))} : {support_data[itemset]}\n")
    sort_and_insert('patterns_close.txt')
    insert_total_count_in_file('patterns_close.txt')

# Run the custom analysis on a specific file
run_custom_analysis('categories.txt')
